In [ ]:
%matplotlib notebook

import time

from matplotlib import pyplot as plt
from matplotlib import colors, cm
import numpy as np

In [ ]:
import qcodes as qc
from qcodes.dataset.experiment_container import new_experiment

from pytopo.qctools import instruments as instools
from pytopo.qctools.instruments import create_inst, add2station

# Init

In [ ]:
%run -i D:/OneDrive/BF2/Code/BF2/BF2/init_station.py
%run -i D:/OneDrive/BF2/Code/BF2/BF2/init_station_cqedVNA.py

In [ ]:
from qcodes.instrument_drivers.rohde_schwarz.SGS100A import RohdeSchwarz_SGS100A
TWPA = instools.create_inst(RohdeSchwarz_SGS100A, 'TWPA',
                            address="TCPIP0::169.254.167.18", force_new_instance=True)
station = add2station(station, TWPA)

## instruments

In [ ]:
vna = station.vna
vna.clear_channels()
vna.add_channel('S21')

# Measurement functions

In [ ]:
def configure_vna_trace(vna, chan_name='S21', **kw):
    trace = getattr(vna.channels, chan_name)
    for k, v in kw.items():
        trace.set(k, v)
    return trace
        

def take_vna_trace_logmag(vna, chan_name, **kw):
    trace = configure_vna_trace(vna, chan_name, **kw)
    vna.rf_on()
    
    f0, f1, nf = trace.start(), trace.stop(), trace.npts()
    fvals = np.linspace(f0, f1, nf)
    
    mag, phase = trace.trace_mag_phase()
    mag = 20*np.log10(mag)
    
    return fvals, mag, phase


# qcodes measurement functions
def measure_vna_trace(vna, chan_name, **trace_cfg):
    sample = qc.config.user.get('current_sample')
    
    trace = configure_vna_trace(vna, 'S21', **trace_cfg)
    
    exp = qc.load_or_create_experiment(
        f'VNA{chan_name}', sample
    )
    
    meas = qc.Measurement(exp, station)
    meas.register_custom_parameter('frequency', unit='Hz', 
                                   paramtype='array')
    meas.register_custom_parameter('magnitude', unit='dB', 
                                   setpoints=('frequency',),
                                   paramtype='array')
    meas.register_custom_parameter('phase', unit='rad', 
                                   setpoints=('frequency',),
                                   paramtype='array')
    with meas.run() as datasaver:
        fvals, mag, phase = take_vna_trace_logmag(vna, chan_name)
        datasaver.add_result(
            ('frequency', fvals),
            ('magnitude', mag),
            ('phase', phase),
        )
    
    return datasaver.dataset


def measure_vna_trace_vs_parameter(vna, chan_name, 
                                   parameter, parameter_vals, 
                                   **trace_cfg):
    sample = qc.config.user.get('current_sample')
    
    trace = configure_vna_trace(vna, 'S21', **trace_cfg)
    
    exp = qc.load_or_create_experiment(
        f'VNA{chan_name}_vs_{parameter.full_name}', sample
    )
    
    meas = qc.Measurement(exp, station)
    meas.register_parameter(parameter)
    meas.register_custom_parameter('frequency', unit='Hz', 
                                   paramtype='array')
    meas.register_custom_parameter('magnitude', unit='dB', 
                                   setpoints=(parameter, 'frequency',),
                                   paramtype='array')
    meas.register_custom_parameter('phase', unit='rad', 
                                   setpoints=(parameter, 'frequency',),
                                   paramtype='array')
    
    with meas.run() as datasaver:
        for v in parameter_vals:
            parameter.set(v)
            fvals, mag, phase = take_vna_trace_logmag(vna, chan_name)
            datasaver.add_result(
                (parameter, v),
                ('frequency', fvals),
                ('magnitude', mag),
                ('phase', phase),
            )

    return datasaver.dataset

# Simple VNA trace

In [1]:
ds = measure_vna_trace(
    vna, 'S21', 
    start=4.5e9, stop=5.0e9, npts=1001,
    power=-55, bandwidth=1e3, avg=10
)

NameError: name 'measure_vna_trace' is not defined

# VNA vs parameter

In [ ]:
ds = measure_vna_trace_vs_parameter(
    vna, 'S21', 
    station.TWPA.power, np.arange(4., 8., 0.1),
#     station.TWPA.frequency, np.arange(7950, 8200, 1.0)*1e6,
    start=4.5e9, stop=5e9, npts=2001,
    power=-55, bandwidth=1e3, avg=10,
)

# Scratch